# Text Mining

## Version 1

In [ ]:
from pdfminer.high_level import extract_text
import os

# Une fonction pour l'extraction du texte dans le pdf
def extract_text_from_pdf(pdf_path):
 
    if not os.path.exists(pdf_path):
        print(f"Erreur : Le fichier '{pdf_path}' n'existe pas. Vérifiez le chemin.")
        return None

    try:
        text = extract_text(pdf_path)
        return text
    except Exception as e:
        print(f"Erreur lors de l'extraction du texte du PDF : {e}")
        return None


pdf_file_path = "article_VO.pdf"

article_text = extract_text_from_pdf(pdf_file_path)

if article_text:
    print(f"Texte extrait :\n{'-'*30}\n{article_text[:500]}...\n{'-'*30}")
else:
    print("Échec de l'extraction du texte. Le nuage de mots ne pourra pas être généré.")

In [ ]:
#important de télecharger le package de mots de la librairie scapy
#  python -m spacy download fr_core_news_sm

In [ ]:
import spacy
from collections import Counter
import re
from wordcloud import WordCloud


# Charger le modèle spaCy pour le français
try:
    nlp = spacy.load("fr_core_news_sm")
except OSError:
    print("Modèle spaCy 'fr_core_news_sm' non trouvé. Veuillez l'installer avec : python -m spacy download fr_core_news_sm")
    exit() # Quitte le script si le modèle n'est pas là

def preprocess_text_for_wordcloud(text):

    """
    Prétraite le texte pour un nuage de mots pertinent en français.
    Mise en minuscules, suppression de la ponctuation, des nombres, des mots vides et lemmatisation.
    """
    doc = nlp(text.lower()) # Convertir en minuscules et traiter

    tokens = []
    for token in doc:
        # Filtrer :
        # - is_punct : Ponctuation
        # - is_space : Espaces, retours chariot
        # - is_stop : Mots vides (articles, prépositions, conjonctions, etc.)
        # - is_alpha : S'assurer que c'est bien une lettre (pas un nombre ou un symbole après filtrage)
        # - len(token.lemma_) > 1 : Éliminer les caractères uniques restants
        if not token.is_punct and not token.is_space and not token.is_stop and token.is_alpha and len(token.lemma_) > 1:
            tokens.append(token.lemma_) # Utiliser la lemmatisation

    return tokens

if article_text:
    processed_tokens = preprocess_text_for_wordcloud(article_text)
    print(f"\nTokens prétraités (aperçu des 50 premiers) :\n{processed_tokens[:50]}...")

    # Calculer la fréquence des mots après prétraitement
    word_counts = Counter(processed_tokens)
    print(f"\n20 mots les plus fréquents après prétraitement :\n{word_counts.most_common(20)}")
else:
    print("Le texte n'a pas pu être prétraité car il n'a pas été extrait du PDF.")

In [ ]:
if processed_tokens:
    text_for_wordcloud = " ".join(processed_tokens)

    # Configurer le WordCloud
    # 'width' et 'height' : Taille de l'image
    # 'background_color' : Couleur de fond
    # 'max_words' : Nombre maximal de mots à afficher
    # 'min_font_size' : Taille minimale des polices
    # 'collocations=False' : Important pour éviter que WordCloud ne regroupe des mots comme "pomme de" s'ils ne sont pas un concept unique
    # 'stopwords' : Même si déjà filtrés, on peut les repasser pour être sûr (WordCloud a aussi sa propre liste)
    #               Pour WordCloud, il faut une liste, pas un set.
    french_stopwords = spacy.lang.fr.stop_words.STOP_WORDS # Utilise la liste de mots vides de spaCy

    wordcloud = WordCloud(width=1200, height=600,
                          background_color='white',
                          max_words=100, # Afficher les 100 mots les plus pertinents
                          min_font_size=10,
                          collocations=False,
                          stopwords=french_stopwords # Appliquer les stopwords ici aussi
                         ).generate(text_for_wordcloud) # Utiliser la chaîne de tokens prétraités

    # Afficher le nuage de mots
    plt.figure(figsize=(15, 8))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off') # Cache les axes
    plt.title("Nuage de mots pertinents de l'article de presse")
    plt.show()
else:
    print("Impossible de générer le nuage de mots car aucun token pertinent n'a été extrait.")

## Version 2

In [ ]:
import sys
!{sys.executable} -m pip install PyPDF2 nltk wordcloud matplotlib
!{sys.executable} -m pip install stanza
import nltk

# Téléchargement des ressources NLTK (une seule fois suffit)
nltk.download("punkt")
nltk.download("stopwords")

stanza.download("fr")
nlp = stanza.Pipeline(
    lang="fr",
    processors="tokenize,mwt,pos,lemma",
    use_gpu=False
)

In [ ]:
import os
import PyPDF2
import stanza
import nltk
from nltk.corpus import stopwords
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Extraire le texte
def extract_text_from_pdf(pdf_path: str) -> str:
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"Le fichier '{pdf_path}' est introuvable.")
    reader = PyPDF2.PdfReader(pdf_path)
    return "".join(page.extract_text() or "" for page in reader.pages)

article_text = extract_text_from_pdf("article_VO.pdf")

# Préparer les listes de filtrage
french_stops = set(stopwords.words("french"))
custom_exclude = {"soi", "leboncoin"}  # mots spécifiques à retirer

# Lemmatisation + filtrage
doc = nlp(article_text)
lemmas = []
for sent in doc.sentences:
    for word in sent.words:
        lemma = word.lemma.lower()
        if (
            word.upos not in ("PUNCT", "SYM", "NUM", "X", "VERB")  # on enlève aussi les verbes
            and len(lemma) > 1
            and lemma not in french_stops
            and lemma not in custom_exclude
        ):
            lemmas.append(lemma)

# Aperçu et top 20
counts = Counter(lemmas)
print("20 lemmes les plus fréquents après filtration :")
for l, c in counts.most_common(20):
    print(f"{l:12s} → {c}")

# Générer le nuage de mots
if lemmas:
    wc = WordCloud(
        width=1200,
        height=600,
        background_color="white",
        max_words=100,
        min_font_size=10,
        collocations=False,
        stopwords=french_stops.union(custom_exclude)
    ).generate(" ".join(lemmas))

    plt.figure(figsize=(15, 8))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title("Nuage de mots")
    plt.show()
else:
    print("Aucun lemme pertinent après filtration.")